In [ ]:
!pip install arch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 916.4/916.4 kB 13.3 MB/s eta 0:00:00


In [ ]:
import warnings
warnings.filterwarnings('ignore')
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from pandas import Timestamp
from statsmodels.tsa.stattools import adfuller
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from datetime import datetime
from sklearn.model_selection import train_test_split
from arch.bootstrap import CircularBlockBootstrap
import yfinance as yf
from scipy.optimize import minimize
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error
from statistics import mean


In [ ]:
# Функция для расчета коэффициента alpha + 1
def calculate_alpha_plus_one(y_true, y_pred):
    R = y_true - y_pred
    R_series = pd.Series(R)
    delta_R = R_series.diff()
    delta_R = delta_R.dropna().values
    R = R[1:]
    cov_delta_R_R = np.cov(delta_R, R)[0, 1]
    var_R = np.var(R)

    # Результат равен (ковариация между delta_R и R - дисперсия R)
    ans = cov_delta_R_R - var_R

    return ans

# Функция для расчета среднеквадратической ошибки
def calculate_mse(y_true, y_pred):
    n = len(y_true)
    ans = np.sum((y_true - y_pred) ** 2) / n
    return ans

# Функция для расчета модифицированной функции потерь
def calculate_modified_loss(y_l_true, y_r_true, y_l_pred, y_r_pred, C, whole_data):
    mse_l = calculate_mse(y_l_true, y_l_pred)
    mse_r = calculate_mse(y_r_true, y_r_pred)

    # Если какое-то из y_l_true, y_l_pred равно нулю, то результат будет NaN
    n_l = len(y_l_true)
    n_r = len(y_r_true)
    sum = n_l + n_r

    alpha_plus_one = calculate_alpha_plus_one(whole_data["y_true"], whole_data["y_pred"])
    ans = 1* mse_l * n_l/sum + 1* mse_r * n_r/sum + C * abs(alpha_plus_one)

    return ans

# Функция для расчета потерь в зависимости от значений gamma_l и gamma_r
def F2_loss(gamma_l, gamma_r, y_l_true, y_r_true, y_l_hat, y_r_hat, C, whole_data):
    y_l_pred = np.full_like(y_l_true, y_l_hat + gamma_l)
    y_r_pred = np.full_like(y_r_true, y_r_hat + gamma_r)

    return calculate_modified_loss(y_l_true, y_r_true, y_l_pred, y_r_pred, C, whole_data)

# Функция для поиска оптимальных значений gamma_l и gamma_r
def find_optimal_gammas(y_l_true, y_r_true, y_l_hat, y_r_hat, C, whole_data):
    opt_result = minimize(
        lambda x: F2_loss(x[0], x[1], y_l_true, y_r_true, y_l_hat, y_r_hat, C, whole_data),
        [0, 0],
        bounds=[(-np.inf, np.inf), (-np.inf, np.inf)],
    )

    # Если оптимизация прошла успешно, возвращаем найденные оптимальные значения gamma_l и gamma_r
    # В противном случае возвращаем 0, 0
    if opt_result.success:
        gamma_l_opt, gamma_r_opt = opt_result.x
    else:
        gamma_l_opt, gamma_r_opt = 0, 0

    return gamma_l_opt, gamma_r_opt

# Класс, представляющий узел дерева решений
class TreeNode:
    def __init__(self, feature=None, threshold=None, left=None, right=None, gamma_l=None, gamma_r=None, y=None):
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.gamma_l = gamma_l
        self.gamma_r = gamma_r
        self.y = y

# Функция для поиска оптимального разделения данных
def find_best_split(df, target_col, whole_data, C = None, max_features = None):
    min_loss = float('inf')
    best_feature = None
    best_threshold = None
    best_gamma_l = None
    best_gamma_r = None

    features = df.drop(target_col, axis = 1).columns

    if max_features is not None:
        max_features = min(max_features, len(features))
        features = np.random.choice(features, max_features, replace = False)

    best_whole_data = whole_data

    for feature in features:
        for threshold in df[feature].unique():
            whole_data_for_pair = whole_data

            # Определяем подмножества данных, которые попадают в левую и правую ветви дерева
            left_mask = df[feature] < threshold
            right_mask = ~left_mask
            if len(df.loc[left_mask, target_col]) < 2 or len(df.loc[right_mask, target_col]) < 2:
                # Если данное разделение приводит к образованию ветви с меньшим количеством данных, чем 2, пропускаем его
                continue

            y_l_hat = df.loc[left_mask, target_col].mean()
            y_r_hat = df.loc[right_mask, target_col].mean()

            whole_data_for_pair['y_pred'].loc[left_mask.index[left_mask]] = y_l_hat
            whole_data_for_pair['y_pred'].loc[right_mask.index[right_mask]] = y_r_hat

            # Находим оптимальные значения gamma_l и gamma_r для данного разделения
            gamma_l, gamma_r = find_optimal_gammas(df.loc[left_mask, target_col], df.loc[right_mask, target_col], y_l_hat, y_r_hat, C, whole_data_for_pair)

            y_l_pred = y_l_hat + gamma_l
            y_r_pred = y_r_hat + gamma_r

            whole_data_for_pair['y_pred'].loc[left_mask.index[left_mask]] = y_l_pred
            whole_data_for_pair['y_pred'].loc[right_mask.index[right_mask]] = y_r_pred

            # Вычисляем потери для данного разделения
            loss = calculate_modified_loss(df.loc[left_mask, target_col], df.loc[right_mask, target_col], y_l_pred, y_r_pred, C, whole_data_for_pair)

            # Если потери меньше текущего минимума, обновляем минимум и запоминаем параметры разделения
            if loss < min_loss:
                min_loss = loss
                best_feature = feature
                best_threshold = threshold
                best_gamma_l = gamma_l
                best_gamma_r = gamma_r
                best_whole_data = whole_data_for_pair
    return best_feature, best_threshold, best_gamma_l, best_gamma_r, min_loss, best_whole_data

# Функция для построения дерева решений
def build_tree(df, target_col, max_depth, min_samples_split, whole_data, C = None, max_features= None, depth = 0):
    if depth == max_depth or len(df) < min_samples_split:
        return TreeNode(y=df[target_col].mean())

    best_feature, best_threshold, best_gamma_l, best_gamma_r, _, best_whole_data = find_best_split(df, target_col, whole_data, C, max_features)
    if best_feature is None:
        return TreeNode(y=df[target_col].mean())

    left_mask = df[best_feature] < best_threshold
    counts = left_mask.value_counts()
    right_mask = ~left_mask

    left_child = build_tree(df[left_mask], target_col, max_depth, min_samples_split, best_whole_data, C, max_features, depth + 1)
    right_child = build_tree(df[right_mask], target_col, max_depth, min_samples_split, best_whole_data, C, max_features, depth + 1)

    return TreeNode(feature = best_feature, threshold = best_threshold, left = left_child, right = right_child, gamma_l = best_gamma_l, gamma_r = best_gamma_r)

# Функция для выполнения прогноза на узле дерева
def predict(node, x):
    if node.y is not None:
        return node.y

    if x[node.feature] < node.threshold:
        return predict(node.left, x) + node.gamma_l
    else:
        return predict(node.right, x) + node.gamma_r

# Класс, реализующий алгоритм Случайного леса
class CustomRandomForest:
    def __init__(self, n_trees, max_depth, min_samples_split, max_features=None, block_size=None, C = None):
        self.trees = [None] * n_trees
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.max_features = max_features
        self.block_size = block_size
        self.C = C

    # Функция для обучения модели
    def fit(self, X, y):
        for i in range(len(self.trees)):
            print('■ FIT FOR TREE ' + str(i+1) + ' OUT OF ' + str(len(self.trees)) + '■')
            df = pd.concat([X, y], axis=1)

            bs = CircularBlockBootstrap(self.block_size, df)
            sample = next(bs.bootstrap(df.shape[0]))[0][0]
            if self.max_features is not None:
                features = X.columns.to_list()
                random_features = np.random.choice(features, self.max_features, replace=False)
                sample = sample[random_features.tolist() + [y.name]]
                y_mean = np.mean(sample[y.name])

                y_pred = np.full_like(sample[y.name], y_mean)

                whole_data = pd.DataFrame({"y_true": sample[y.name], "y_pred": y_pred})

            self.trees[i] = build_tree(sample, y.name, self.max_depth, self.min_samples_split, whole_data, self.C)

    # Функция для выполнения прогноза
    def predict(self, X):
        predictions = np.zeros(len(X))
        for tree in self.trees:
            y_pred = np.array([predict(tree, x) for _, x in X.iterrows()])
            predictions += y_pred
        return predictions / len(self.trees)

def walk_forward_validation(data, window_size):

    splits = []
    step_size = int(window_size*0.35)
    for i in range(0, len(data) - window_size + 1, step_size):
        data_part = data[i: i + window_size]
        splits.append(data_part)
    return splits

In [ ]:
dataset = pd.read_csv('/content/weather.csv', delimiter = ',')
dataset

,date,p (mbar),T (degC),Tpot (K),Tdew (degC),rh (%),VPmax (mbar),VPact (mbar),VPdef (mbar),sh (g/kg),...,wv (m/s),max. wv (m/s),wd (deg),rain (mm),raining (s),SWDR (W/m�),PAR (�mol/m�/s),max. PAR (�mol/m�/s),Tlog (degC),OT
0,2020-01-01 00:10:00,1008.89,0.71,273.18,-1.33,86.1,6.43,5.54,0.89,3.42,...,1.02,1.60,224.3,0.0,0.0,0.0,0.0,0.0,11.45,428.1
1,2020-01-01 00:20:00,1008.76,0.75,273.22,-1.44,85.2,6.45,5.49,0.95,3.39,...,0.43,0.84,206.8,0.0,0.0,0.0,0.0,0.0,11.51,428.0
2,2020-01-01 00:30:00,1008.66,0.73,273.21,-1.48,85.1,6.44,5.48,0.96,3.39,...,0.61,1.48,197.1,0.0,0.0,0.0,0.0,0.0,11.60,427.6
3,2020-01-01 00:40:00,1008.64,0.37,272.86,-1.64,86.3,6.27,5.41,0.86,3.35,...,1.11,1.48,206.4,0.0,0.0,0.0,0.0,0.0,11.70,430.0
4,2020-01-01 00:50:00,1008.61,0.33,272.82,-1.50,87.4,6.26,5.47,0.79,3.38,...,0.49,1.40,209.6,0.0,0.0,0.0,0.0,0.0,11.81,432.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52691,2020-12-31 23:20:00,978.32,2.28,277.16,-0.80,80.0,7.20,5.76,1.44,3.67,...,0.73,1.40,180.6,0.0,0.0,0.0,0.0,0.0,13.40,433.0
52692,2020-12-31 23:30:00,978.30,2.13,277.01,-0.43,83.1,7.12,5.92,1.20,3.77,...,0.43,0.82,174.0,0.0,0.0,0.0,0.0,0.0,13.42,439.6
52693,2020-12-31 23:40:00,978.26,1.99,276.88,-0.71,82.2,7.05,5.80,1.26,3.69,...,0.38,0.76,248.9,0.0,0.0,0.0,0.0,0.0,13.45,435.2
52694,2020-12-31 23:50:00,978.26,2.07,276.95,-0.77,81.4,7.09,5.77,1.32,3.68,...,0.57,1.07,196.6,0.0,0.0,0.0,0.0,0.0,13.47,433.9


In [ ]:
dataset = dataset[['date', 'T (degC)', 'p (mbar)', 'rh (%)', 'sh (g/kg)', 'wv (m/s)', 'rain (mm)', ]]

In [ ]:
dataset['date'] = pd.to_datetime(dataset['date'], format='%Y-%m-%d')


dataset.set_index('date', inplace=True)
dataset = dataset[dataset.index <= '2020-01-11']

dataset

,T (degC),p (mbar),rh (%),sh (g/kg),wv (m/s),rain (mm)
date,,,,,,
2020-01-01 00:10:00,0.71,1008.89,86.1,3.42,1.02,0.0
2020-01-01 00:20:00,0.75,1008.76,85.2,3.39,0.43,0.0
2020-01-01 00:30:00,0.73,1008.66,85.1,3.39,0.61,0.0
2020-01-01 00:40:00,0.37,1008.64,86.3,3.35,1.11,0.0
2020-01-01 00:50:00,0.33,1008.61,87.4,3.38,0.49,0.0
...,...,...,...,...,...,...
2020-01-10 23:20:00,4.33,999.87,84.4,4.38,1.51,0.0
2020-01-10 23:30:00,4.25,1000.01,82.1,4.24,2.54,0.0
2020-01-10 23:40:00,4.13,1000.23,79.3,4.06,2.54,0.0


In [ ]:
from sklearn.metrics import mean_absolute_error, r2_score

window_size = 100  # выберите размер окна

lagged_dataset = dataset.shift(1)
lagged_dataset['T (degC)'] = dataset['T (degC)']

lagged_dataset = lagged_dataset.dropna()

data_full = lagged_dataset

splits = walk_forward_validation(data_full, window_size)

train_size = 0.8

final_mse_train = []
final_mse_test = []

# Инициализация массивов для хранения метрик
standart_final_mae_train = []
standart_final_r2_train = []
standart_final_mae_test = []
standart_final_r2_test = []

custom_final_mae_train = []
custom_final_r2_train = []
custom_final_mae_test = []
custom_final_r2_test = []

standart_final_mse_train = []
standart_final_mse_test = []
custom_final_mse_train = []
custom_final_mse_test = []

intervals = []

C = 0.75
for data in splits:
  print(data.index[0],  data.index[-1])

  X = data.drop(columns=['T (degC)'])
  y = data['T (degC)']


  X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=train_size, shuffle=False)

  scaler_X = MinMaxScaler()
  scaler_y = MinMaxScaler()

  X_train_scaled = scaler_X.fit_transform(X_train)
  y_train_scaled = scaler_y.fit_transform(y_train.values.reshape(-1, 1))

  X_test_scaled = scaler_X.transform(X_test)
  y_test_scaled = scaler_y.transform(y_test.values.reshape(-1, 1))

  X_train_scaled_df = pd.DataFrame(X_train_scaled, columns=X_train.columns, index=X_train.index)
  y_train_scaled_df = pd.Series(y_train_scaled.flatten(), index=y_train.index, name=y_train.name)
  X_test_scaled_df = pd.DataFrame(X_test_scaled, columns=X_test.columns, index=X_test.index)
  y_test_scaled_df = pd.Series(y_test_scaled.flatten(), index=y_test.index, name=y_test.name)


  model = CustomRandomForest(n_trees=10, max_depth=6, min_samples_split=3, max_features=3, block_size=4, C = C)

  # Обучение модели с использованием масштабированных данных
  model.fit(X_train_scaled_df, y_train_scaled_df)
  # Прогноз на тренировочном наборе
  y_train_pred_scaled = model.predict(X_train_scaled_df)
  # Преобразование y_pred_scaled в 2D массив, потому что метод inverse_transform от MinMaxScaler ожидает 2D данные
  y_train_pred_scaled_2D = np.array(y_train_pred_scaled).reshape(-1, 1)
  # Обратное преобразование прогнозов
  y_train_pred = scaler_y.inverse_transform(y_train_pred_scaled_2D)
  # Преобразование y_train_pred в 1D массив
  y_train_pred = y_train_pred.flatten()


  y_pred_scaled = model.predict(X_test_scaled_df)
  y_pred_scaled_2D = np.array(y_pred_scaled).reshape(-1, 1)
  y_pred = scaler_y.inverse_transform(y_pred_scaled_2D)
  y_pred = y_pred.flatten()

  # Расчет среднеквадратичной ошибки на тренировочном наборе
  mse_train = mean_squared_error(y_train, y_train_pred)

  # Расчет среднеквадратичной ошибки на тестовом наборе
  mse_test = mean_squared_error(y_test, y_pred)
  print('MSE тест Разраб. RF: C = ' + str(Ci) + ' ' + str(mse_test))

  # Расчет MAE и R2 для тренировочного набора
  mae_train = mean_absolute_error(y_train, y_train_pred)
  r2_train = r2_score(y_train, y_train_pred)

  # Расчет MAE и R2 для тестового набора
  mae_test = mean_absolute_error(y_test, y_pred)
  r2_test = r2_score(y_test, y_pred)
  print('MAE CustomRandomForest тест: C = ' + str(Ci) + ' ' + str(mae_test))
  print('R2 CustomRandomForest тест: C = ' + str(Ci) + ' ' + str(r2_test))

  custom_final_mse_train.append(mse_train)
  custom_final_mse_test.append(mse_test)

  custom_final_mae_train.append(mae_train)
  custom_final_r2_train.append(r2_train)
  custom_final_mae_test.append(mae_test)
  custom_final_r2_test.append(r2_test)

  df_pred = pd.DataFrame({
      'Actual': pd.concat([y_train, y_test]),
      'Predicted_CustomRandomForest': np.concatenate([y_train_pred, y_pred]),
      'Predicted_RandomForestRegressor': np.nan,  # Инициализация колонки, заполнение будет позже
  })


  #ТЕПЕРЬ СТАНДАРТНЫЙ АЛГОРИТМ
  model = RandomForestRegressor(n_estimators=10, max_depth=6, max_features=3, min_samples_split=3)
  model.fit(X_train_scaled_df, y_train_scaled_df)

  y_train_pred_scaled_rf = model.predict(X_train_scaled_df)
  y_train_pred_scaled_2D_rf = np.array(y_train_pred_scaled_rf).reshape(-1, 1)
  y_train_pred_rf = scaler_y.inverse_transform(y_train_pred_scaled_2D_rf)
  y_train_pred_rf = y_train_pred_rf.flatten()

  y_pred_scaled_rf = model.predict(X_test_scaled_df)
  y_pred_scaled_2D_rf = np.array(y_pred_scaled_rf).reshape(-1, 1)
  y_pred_rf = scaler_y.inverse_transform(y_pred_scaled_2D_rf)
  y_pred_rf = y_pred_rf.flatten()

  standart_mse_train = mean_squared_error(y_train, y_train_pred_rf)

  standart_mse_test = mean_squared_error(y_test, y_pred_rf)
  print('\nMSE тест RandomForestRegressor:' + str(standart_mse_test))

  # Расчет MAE и R2 для тренировочного набора
  standart_mae_train = mean_absolute_error(y_train, y_train_pred)
  standart_r2_train = r2_score(y_train, y_train_pred)

  # Расчет MAE и R2 для тестового набора
  standart_mae_test = mean_absolute_error(y_test, y_pred_rf)
  standart_r2_test = r2_score(y_test, y_pred_rf)
  print('MAE RandomForestRegressor тест: C = ' + str(Ci) + ' ' + str(standart_mae_test))
  print('R2 RandomForestRegressor тест: C = ' + str(Ci) + ' ' + str(standart_r2_test))

  if standart_mse_test > mse_test:
    intervals.append((pd.to_datetime(data.index[0]), pd.to_datetime(data.index[-1])),)
    print('лучше')

  standart_final_mse_train.append(standart_mse_train)
  standart_final_mse_test.append(standart_mse_test)

  standart_final_mae_train.append(standart_mae_train)
  standart_final_r2_train.append(standart_r2_train)
  standart_final_mae_test.append(standart_mae_test)
  standart_final_r2_test.append(standart_r2_test)


  # Добавить прогнозы RandomForest в DataFrame
  df_pred.loc[y_train.index, 'Predicted_RandomForestRegressor'] = y_train_pred_rf
  df_pred.loc[y_test.index, 'Predicted_RandomForestRegressor'] = y_pred_rf


  plt.figure(figsize=(15,10))  # Увеличение размера графика

  # Seaborn имеет набор предопределенных тем для оформления своих графиков. Вы можете задать одну из этих тем, используя функцию sns.set_theme().
  sns.set_theme(style="darkgrid")  # Добавить сетку и темный фон

  # Задайте голубоватый цвет фона
  sns.set(rc={"axes.facecolor":"#E6F0F3", "figure.facecolor":"#E6F0F3"})  # Слегка голубой фон

  # Построить график
  sns.lineplot(data=df_pred)

  # Добавьте вертикальную линию для разделения тренировочных и тестовых данных
  plt.axvline(df_pred.loc[y_train.index].index[-1], color='gray', linestyle='--')

  # Добавьте текст 'порог' возле линии
  plt.text(df_pred.loc[y_train.index].index[-1], df_pred['Actual'].min(), 'тренировка/тест',
          rotation=90, horizontalalignment='right', verticalalignment='bottom')

  plt.title('Актуальное vs Прогноз', fontsize=20)
  plt.xlabel('Дата', fontsize=15)
  plt.ylabel('Высокий', fontsize=15)
  plt.xticks(fontsize=13)
  plt.yticks(fontsize=13)
  plt.show()


print('mse_avg_test стандарт = ' + str(mean(standart_final_mse_test)))
print('mae_avg_test стандарт = ' + str(mean(standart_final_mae_test)))
print('r2_avg_test стандарт = ' + str(mean(standart_final_r2_test)))

print('mse_avg_test пользовательский = ' + str(mean(custom_final_mse_test)))
print('mae_avg_test пользовательский = ' + str(mean(custom_final_mae_test)))
print('r2_avg_test пользовательский = ' + str(mean(custom_final_r2_test)))
